# Measuring the Local Health Impact of Animal Agriculture

We're going to use publicly accessible data from the USDA and various state health organizations to see what impact, if any, animal facilities have on the health of people living nearby. Specifically, we going to look at the prevalence of bacterial infections of:
 - STEC (Shiga toxin-producing E. coli)
 - MRSA (Methicillin-resistent Styphylococcus aureus)
 - Campylobacter
 - Salmonella

In [1]:
import math
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
#import plotly.graph_objects as go
import plotly
from plotly.offline import iplot, init_notebook_mode

init_notebook_mode(connected=True)

def plot(figure):
    plotly.offline.iplot(figure)

Let's first import a full data set from the USDA Census (2017), which gathers all sorts of statistics from farms around the country. This data is not completely deaggregated, so we can't see down to an individual farm level, but we can see down to a county-by-county level, which should be enough for our purposes. It's available as a single 1GB CSV file.

In [2]:
# df = pd.read_csv('2017_cdqt_data.txt', sep='\t', skiprows=(0), header=(0), low_memory=False)
agCensus = pd.read_csv('2017_cdqt_data.txt', sep='\t', header=0, low_memory=False)

Let's take a look at how this data is organized:

In [3]:
agCensus.columns

Index(['CENSUS_CHAPTER', 'CENSUS_TABLE', 'CENSUS_ROW', 'CENSUS_COLUMN',
       'SECTOR_DESC', 'SHORT_DESC', 'COMMODITY_DESC', 'AGG_LEVEL_DESC',
       'STATE_FIPS_CODE', 'STATE_ALPHA', 'STATE_NAME', 'COUNTY_CODE',
       'COUNTY_NAME', 'DOMAINCAT_DESC', 'VALUE'],
      dtype='object')

Let's narrow down our data just to Texas so we can get a better look at it, and verify we have county-level data:

In [4]:
agCensusTexas = agCensus[agCensus['STATE_ALPHA'] == "TX"]
agCensusTexas['COUNTY_NAME'].unique()

array([nan, 'ANDERSON', 'ANDREWS', 'ANGELINA', 'ARANSAS', 'ARCHER',
       'ARMSTRONG', 'ATASCOSA', 'AUSTIN', 'BAILEY', 'BANDERA', 'BASTROP',
       'BAYLOR', 'BEE', 'BELL', 'BEXAR', 'BLANCO', 'BORDEN', 'BOSQUE',
       'BOWIE', 'BRAZORIA', 'BRAZOS', 'BREWSTER', 'BRISCOE', 'BROOKS',
       'BROWN', 'BURLESON', 'BURNET', 'CALDWELL', 'CALHOUN', 'CALLAHAN',
       'CAMERON', 'CAMP', 'CARSON', 'CASS', 'CASTRO', 'CHAMBERS',
       'CHEROKEE', 'CHILDRESS', 'CLAY', 'COCHRAN', 'COKE', 'COLEMAN',
       'COLLIN', 'COLLINGSWORTH', 'COLORADO', 'COMAL', 'COMANCHE',
       'CONCHO', 'COOKE', 'CORYELL', 'COTTLE', 'CRANE', 'CROCKETT',
       'CROSBY', 'CULBERSON', 'DALLAM', 'DALLAS', 'DAWSON', 'DEAF SMITH',
       'DELTA', 'DENTON', 'DE WITT', 'DICKENS', 'DIMMIT', 'DONLEY',
       'DUVAL', 'EASTLAND', 'ECTOR', 'EDWARDS', 'ELLIS', 'EL PASO',
       'ERATH', 'FALLS', 'FANNIN', 'FAYETTE', 'FISHER', 'FLOYD', 'FOARD',
       'FORT BEND', 'FRANKLIN', 'FREESTONE', 'FRIO', 'GAINES',
       'GALVESTON', 'GARZ

It looks like some data is not affiliated to particular counties, so let's filter all of that out, and also see what this "SECTOR_DESC" field is about:

In [5]:
agCensusTexas2 = agCensusTexas[agCensusTexas['COUNTY_NAME'].str.contains('.+', na=False)]
agCensusTexas2['SECTOR_DESC'].unique()

array(['ECONOMICS', 'CROPS', 'ANIMALS & PRODUCTS', 'ENVIRONMENTAL',
       'DEMOGRAPHICS'], dtype=object)

We care only about "ANIMALS & PRODUCTS", so we'll filter everything else out, and see what the "SHORT_DESC" is about:

In [6]:
agCensusTexas3 = agCensusTexas2[agCensusTexas2['SECTOR_DESC'] == 'ANIMALS & PRODUCTS']
agCensusTexas3['SHORT_DESC'].unique()

array(['ANIMAL TOTALS, INCL PRODUCTS - SALES, MEASURED IN $',
       'CATTLE, INCL CALVES - OPERATIONS WITH INVENTORY',
       'CATTLE, INCL CALVES - INVENTORY',
       'CATTLE, COWS, BEEF - OPERATIONS WITH INVENTORY',
       'CATTLE, COWS, BEEF - INVENTORY',
       'CATTLE, COWS, MILK - OPERATIONS WITH INVENTORY',
       'CATTLE, COWS, MILK - INVENTORY',
       'CATTLE, INCL CALVES - OPERATIONS WITH SALES',
       'CATTLE, INCL CALVES - SALES, MEASURED IN HEAD',
       'HOGS - OPERATIONS WITH INVENTORY', 'HOGS - INVENTORY',
       'HOGS - OPERATIONS WITH SALES', 'HOGS - SALES, MEASURED IN HEAD',
       'SHEEP, INCL LAMBS - OPERATIONS WITH INVENTORY',
       'SHEEP, INCL LAMBS - INVENTORY',
       'CHICKENS, LAYERS - OPERATIONS WITH INVENTORY',
       'CHICKENS, LAYERS - INVENTORY',
       'CHICKENS, BROILERS - OPERATIONS WITH SALES',
       'CHICKENS, BROILERS - SALES, MEASURED IN HEAD',
       'ANIMAL TOTALS, INCL PRODUCTS - OPERATIONS WITH SALES',
       'POULTRY TOTALS, INCL EGGS -

OK, now we've gotten somewhere! They have data on different animals. Let's start with cows, by filtering for "CATTLE, INCL CALVES - INVENTORY", and checking out what the "DOMAINCAT_DESC" field is about:

In [7]:
texasCattleInventory = agCensusTexas3[agCensusTexas3['SHORT_DESC'] == 'CATTLE, INCL CALVES - INVENTORY']
texasCattleInventory['DOMAINCAT_DESC'].unique()

array([nan, 'INVENTORY OF CATTLE, INCL CALVES: (1 TO 9 HEAD)',
       'INVENTORY OF CATTLE, INCL CALVES: (10 TO 19 HEAD)',
       'INVENTORY OF CATTLE, INCL CALVES: (20 TO 49 HEAD)',
       'INVENTORY OF CATTLE, INCL CALVES: (50 TO 99 HEAD)',
       'INVENTORY OF CATTLE, INCL CALVES: (100 TO 199 HEAD)',
       'INVENTORY OF CATTLE, INCL CALVES: (200 TO 499 HEAD)',
       'INVENTORY OF CATTLE, INCL CALVES: (500 OR MORE HEAD)'],
      dtype=object)

OK, so they  have broken up cattle into buckets, presumably based on the size of the farm. Let's look at a particular county to see exactly what this means:

In [8]:
texasCattleInventory[texasCattleInventory['COUNTY_NAME']=='ANDERSON']

,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,CENSUS_COLUMN,SECTOR_DESC,SHORT_DESC,COMMODITY_DESC,AGG_LEVEL_DESC,STATE_FIPS_CODE,STATE_ALPHA,STATE_NAME,COUNTY_CODE,COUNTY_NAME,DOMAINCAT_DESC,VALUE
129706,2,1,42,2,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,1.0,ANDERSON,NaN,"65,048"
1453335,2,11,3,2,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,1.0,ANDERSON,NaN,"65,048"
1460285,2,11,7,2,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,1.0,ANDERSON,"INVENTORY OF CATTLE, INCL CALVES: (1 TO 9 HEAD)","1,502"
1467220,2,11,11,2,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,1.0,ANDERSON,"INVENTORY OF CATTLE, INCL CALVES: (10 TO 19 HEAD)","3,274"
1474127,2,11,15,2,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,1.0,ANDERSON,"INVENTORY OF CATTLE, INCL CALVES: (20 TO 49 HEAD)","9,698"
1481010,2,11,19,2,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,1.0,ANDERSON,"INVENTORY OF CATTLE, INCL CALVES: (50 TO 99 HEAD)","12,186"
1487729,2,11,23,2,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,1.0,ANDERSON,"INVENTORY OF CATTLE, INCL CALVES: (100 TO 199 ...","8,169"
1494202,2,11,27,2,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,1.0,ANDERSON,"INVENTORY OF CATTLE, INCL CALVES: (200 TO 499 ...","9,292"
1500121,2,11,31,2,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,1.0,ANDERSON,"INVENTORY OF CATTLE, INCL CALVES: (500 OR MORE...","20,927"


The number we want (total cattle per county), seems to be in CENSUS_CHAPTER=2, CENSUS_TABLE=1, so let's filter for this, and as a sanity check we'll make sure we have the same number of rows as we do counties in Texas (254):

In [9]:
texasCattle = texasCattleInventory[(texasCattleInventory['CENSUS_CHAPTER']==2) & (texasCattleInventory['CENSUS_TABLE']==1)]
texasCattle.count()

CENSUS_CHAPTER     254
CENSUS_TABLE       254
CENSUS_ROW         254
CENSUS_COLUMN      254
SECTOR_DESC        254
SHORT_DESC         254
COMMODITY_DESC     254
AGG_LEVEL_DESC     254
STATE_FIPS_CODE    254
STATE_ALPHA        254
STATE_NAME         254
COUNTY_CODE        254
COUNTY_NAME        254
DOMAINCAT_DESC       0
VALUE              254
dtype: int64

Perfect! Let's clean up this data a little bit, though. "COUNTY_CODE" should presumably be an integer, so let's recast it. This will be useful when plotting data on a map, in which counties are referred to by an integer FIPS code.  

In [10]:
texasCattle['COUNTY_CODE'] = texasCattle['COUNTY_CODE'].astype(int)
texasCattle['FIPS'] = (texasCattle['COUNTY_CODE'] + 1000*48).apply(str)
texasCattle.set_index('FIPS', inplace=True, drop=False)
texasCattle['VALUE'] = texasCattle['VALUE'].apply(lambda s: s.replace(',', ''))
texasCattle['VALUE'] = texasCattle['VALUE'].astype(int)
texasCattle['COUNTY_CODE'].unique()
#texasCattle['VALUE']
texasCattle

,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,CENSUS_COLUMN,SECTOR_DESC,SHORT_DESC,COMMODITY_DESC,AGG_LEVEL_DESC,STATE_FIPS_CODE,STATE_ALPHA,STATE_NAME,COUNTY_CODE,COUNTY_NAME,DOMAINCAT_DESC,VALUE,FIPS
FIPS,,,,,,,,,,,,,,,,
48001,2,1,42,2,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,1,ANDERSON,NaN,65048,48001
48003,2,1,42,3,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,3,ANDREWS,NaN,11405,48003
48005,2,1,42,4,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,5,ANGELINA,NaN,19274,48005
48007,2,1,42,5,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,7,ARANSAS,NaN,2766,48007
48009,2,1,42,6,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,9,ARCHER,NaN,78771,48009
48011,2,1,42,7,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,11,ARMSTRONG,NaN,32511,48011
48013,2,1,42,8,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,13,ATASCOSA,NaN,76451,48013
48015,2,1,42,9,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,15,AUSTIN,NaN,63190,48015
48017,2,1,42,10,ANIMALS & PRODUCTS,"CATTLE, INCL CALVES - INVENTORY",CATTLE,COUNTY,48,TX,TEXAS,17,BAILEY,NaN,130261,48017


OK, now let's create some auxilliary lists to build a Plotly map of our data to get an idea of how it looks:

In [11]:
# Code to map increasing list of values to RGB colors
# start is a list of initial r, g, b values
# end is a list of final r, g, b values
# Color is linearly interpolated

def valsToColors(values, start, end):
    maxValue = values[-1]
    minValue = values[0]
    factors = list(map(lambda value: (value - minValue) / (maxValue - minValue), values))
    reds   = list(map(lambda t: int(round((1-t)*start[0] + t*end[0])), factors));
    greens = list(map(lambda t: int(round((1-t)*start[1] + t*end[1])), factors));
    blues  = list(map(lambda t: int(round((1-t)*start[2] + t*end[2])), factors));
    rgbs = [f'rgb({r}, {g}, {b})' for (r,g,b) in zip(reds, greens, blues)]
    return rgbs

In [12]:
####fips = list(texasCattle['COUNTY_CODE'].apply(str))  # Read off county codes
####fips = list(map(lambda s: s.zfill(3), fips))        # Prefix leading zeros
####fips = list(map(lambda code: '48' + code, fips))    # Prefix Texas state code 48 
fips = list(texasCattle['FIPS'])
values = list(texasCattle['VALUE'])
####values = list(map(lambda s: s.replace(',',''), values))
####values = list(map(int, values))
endpts = list(np.mgrid[min(values):max(values):7j])
colorscale = valsToColors(endpts, [200,255,0], [255,50,0])
endpts.pop(0)
endpts.pop(-1)

fig = ff.create_choropleth(fips=fips, values=values, scope=['TX'],
                           binning_endpoints=endpts, colorscale=colorscale,
                           county_outline={'color': 'rgb(0,0,0)', 'width': 0.5},
                           legend_title='Number of Cattle (including calves)')

fig.layout.template = None
fig.show()

/usr/lib/python3.7/site-packages/pandas/core/frame.py:7116: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




Now let's load STEC data for Texas obtained from the Texas DSHS website (we had to extract it using Tabula, since it was only available in PDF form):

In [13]:
texasSTEC = pd.read_csv('Texas_STEC_By_County.csv', sep=',', header=0)

Let's take a quick look at the data. Unlike our previous data set (which had many nested chapters, tables, etc.) this is a single table, so no need to dig around for what we're looking for. "Count" is the number of reported STEC incidents in the given county that year. "IR" is the number of cases per 100,000 residents (i.e. population adjusted):

In [14]:
texasSTEC

,County,2008 Count,2008 IR,2009 Count,2009 IR,2010 Count,2010 IR,2011 Count,2011 IR,2012 Count,...,2013 Count,2013 IR,2014 Count,2014 IR,2015 Count,2015 IR,2016 Count,2016 IR,2017 Count,2017 IR
0,Anderson,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,0,0.0,1,1.6,1,1.6
1,Andrews,1,7.2,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,1,6.2,0,0.0,2,12.0
2,Angelina,3,3.6,0,0.0,1,1.2,5,5.8,1,...,3,3.5,0,0.0,0,0.0,0,0.0,1,1.1
3,Aransas,0,0.0,0,0.0,0,0.0,0,0.0,0,...,2,6.9,1,4.1,0,0.0,0,0.0,0,0.0
4,Archer,0,0.0,0,0.0,0,0.0,0,0.0,1,...,1,10.4,3,31.9,1,10.5,0,0.0,1,10.4
5,Armstrong,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
6,Atascosa,0,0.0,2,4.5,0,0.0,1,2.1,0,...,0,0.0,9,18.3,0,0.0,1,1.9,4,7.6
7,Austin,0,0.0,0,0.0,0,0.0,2,6.8,1,...,0,0.0,0,0.0,0,0.0,1,3.0,1,3.0
8,Bailey,0,0.0,0,0.0,0,0.0,1,15.9,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
9,Bandera,0,0.0,0,0.0,2,9.4,1,4.6,0,...,2,9.0,1,4.5,0,0.0,1,4.3,0,0.0


We can see that the data is quite sparse: incidents of STEC are relatively rare in a given year. This means noise will be a problem. We can alleviate this somewhat by taking an average over all the available years.

We'd like to see if there is a causal link between the number of cattle in a county and cases of STEC. We can't measure causation, so instead we try to measure correlation, but it's important we choose our variables correctly, based on some kind of abstract model of what's going on. Should we be using the incidence rate per 100,000 residents, or the total numbere of cases as our dependent variable? Well, a naive model says that if most transmissions are from some external source (like animals) to humans, and not human to human, then we ought to expect the number of cases to be linear in human population, all else being equal. The intuition here is that we imagine individual bacterium dispersed through the environment which humans are moving around in. Doubling the number of humans doubles the chance that an individual bacterium will encounter a person, and thus doubles the probability that that bacterium will be reponsible for a human being infected.  

In [15]:
countyInfo = pd.read_csv('US_County_Info.csv', sep=',', header=0)

In [16]:
countyInfo.set_index('FIPS', inplace=True, drop=False)

In [17]:
countyInfo[countyInfo['State']=='TX']

,State,FIPS,County,County Seat(s),Population\n(2010),Land Area\nkm²,Land Area\nmi²,Water Area\nkm²,Water Area\nmi²,Total Area\nkm²,Total Area\nmi²,Latitude,Longitude
FIPS,,,,,,,,,,,,,
48001,TX,48001,Anderson,Palestine,58458,2752.126,1062.602,39.884,15.399,2792.010,1078.001,+31.841266°,–95.661744°
48003,TX,48003,Andrews,Andrews,14786,3886.830,1500.713,0.957,0.37,3887.787,1501.083,+32.312258°,–102.640206°
48005,TX,48005,Angelina,Lufkin,86771,2066.235,797.778,173.343,66.928,2239.579,864.706,+31.251951°,–94.611056°
48007,TX,48007,Aransas,Rockport,23158,652.869,252.074,714.621,275.917,1367.490,527.991,+28.104225°,–96.977983°
48009,TX,48009,Archer,Archer City,9054,2339.044,903.110,57.778,22.308,2396.821,925.418,+33.616305°,–98.687267°
48011,TX,48011,Armstrong,Claude,1901,2354.582,909.109,12.22,4.718,2366.802,913.827,+34.964179°,–101.356636°
48013,TX,48013,Atascosa,Jourdanton,44911,3158.605,1219.544,4.996,1.929,3163.601,1221.473,+28.894296°,–98.528187°
48015,TX,48015,Austin,Bellville,28417,1674.449,646.508,25.577,9.875,1700.026,656.383,+29.891901°,–96.270170°
48017,TX,48017,Bailey,Muleshoe,7165,2141.395,826.797,1.772,0.684,2143.167,827.481,+34.067521°,–102.830345°


In [18]:
def tx_fips(county):
    fips = list(countyInfo[(countyInfo['County'].str.upper()==county.upper()) & (countyInfo['State']=='TX')]['FIPS'])[0]
    return str(fips)

In [19]:
texasSTEC['FIPS'] = [tx_fips(county) for county in list(texasSTEC['County'])]
texasSTEC.set_index('FIPS', inplace=True, drop=False)
texasSTEC

,County,2008 Count,2008 IR,2009 Count,2009 IR,2010 Count,2010 IR,2011 Count,2011 IR,2012 Count,...,2013 IR,2014 Count,2014 IR,2015 Count,2015 IR,2016 Count,2016 IR,2017 Count,2017 IR,FIPS
FIPS,,,,,,,,,,,,,,,,,,,,,
48001,Anderson,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,1,1.6,1,1.6,48001
48003,Andrews,1,7.2,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,1,6.2,0,0.0,2,12.0,48003
48005,Angelina,3,3.6,0,0.0,1,1.2,5,5.8,1,...,3.5,0,0.0,0,0.0,0,0.0,1,1.1,48005
48007,Aransas,0,0.0,0,0.0,0,0.0,0,0.0,0,...,6.9,1,4.1,0,0.0,0,0.0,0,0.0,48007
48009,Archer,0,0.0,0,0.0,0,0.0,0,0.0,1,...,10.4,3,31.9,1,10.5,0,0.0,1,10.4,48009
48011,Armstrong,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,48011
48013,Atascosa,0,0.0,2,4.5,0,0.0,1,2.1,0,...,0.0,9,18.3,0,0.0,1,1.9,4,7.6,48013
48015,Austin,0,0.0,0,0.0,0,0.0,2,6.8,1,...,0.0,0,0.0,0,0.0,1,3.0,1,3.0,48015
48017,Bailey,0,0.0,0,0.0,0,0.0,1,15.9,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,48017


In [20]:
def agg_rate(fips):
    agg_rate = 0
    for i in range(2008,2018):
        agg_rate += texasSTEC[str(i)+' IR'][fips]
    return agg_rate

In [21]:
texasSTEC['AGG_RATE'] = list(map(agg_rate, texasSTEC.index))
texasSTEC

,County,2008 Count,2008 IR,2009 Count,2009 IR,2010 Count,2010 IR,2011 Count,2011 IR,2012 Count,...,2014 Count,2014 IR,2015 Count,2015 IR,2016 Count,2016 IR,2017 Count,2017 IR,FIPS,AGG_RATE
FIPS,,,,,,,,,,,,,,,,,,,,,
48001,Anderson,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,1,1.6,1,1.6,48001,3.2
48003,Andrews,1,7.2,0,0.0,0,0.0,0,0.0,0,...,0,0.0,1,6.2,0,0.0,2,12.0,48003,25.4
48005,Angelina,3,3.6,0,0.0,1,1.2,5,5.8,1,...,0,0.0,0,0.0,0,0.0,1,1.1,48005,16.4
48007,Aransas,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1,4.1,0,0.0,0,0.0,0,0.0,48007,11.0
48009,Archer,0,0.0,0,0.0,0,0.0,0,0.0,1,...,3,31.9,1,10.5,0,0.0,1,10.4,48009,73.6
48011,Armstrong,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,48011,0.0
48013,Atascosa,0,0.0,2,4.5,0,0.0,1,2.1,0,...,9,18.3,0,0.0,1,1.9,4,7.6,48013,34.4
48015,Austin,0,0.0,0,0.0,0,0.0,2,6.8,1,...,0,0.0,0,0.0,1,3.0,1,3.0,48015,16.2
48017,Bailey,0,0.0,0,0.0,0,0.0,1,15.9,0,...,0,0.0,0,0.0,0,0.0,0,0.0,48017,15.9


In [22]:
x_vals = np.asarray(texasCattle.loc[texasSTEC['FIPS']]['VALUE'])
x_vals = x_vals.astype('float64')
#x_vals = x_vals/max(x_vals)
y_vals = np.asarray(texasSTEC.loc[texasSTEC['FIPS']]['AGG_RATE'])
y_vals = y_vals.astype('float64')

In [23]:
#plot = go.Figure(data=go.Scatter(x=x_vals, y=y_vals, mode='markers'))
#plot.show()
trace = {'x' : x_vals, 'y' : y_vals, 'mode' : 'markers'}
figure = {'data': [trace], 'layout': {'title': 'STEC Incidence Rate versus Cattle Population'}}
plot(figure)

In [24]:
def errors(m, b, xs, ys):
    return ys - (m*xs + b)

def MSE(m, b, xs, ys):
    return sum(errors(m, b, xs, ys)**2)/xs.size

def dMSE_dm(m, b, xs, ys):
    return -2*sum(xs*errors(m, b, xs, ys))/xs.size
    
def dMSE_db(m, b, xs, ys):
    return -2*sum(errors(m, b, xs, ys))/xs.size

def RMSE(m, b, xs, ys):
    return math.sqrt(MSE(m,b, xs, ys))   

In [25]:
def step(m, b, x_values, y_values, learn_rate):
    m_new = m - learn_rate * dMSE_dm(m, b, x_values, y_values)
    b_new = b - learn_rate * dMSE_db(m,b, x_values, y_values)
    return (m_new, b_new)

def learn(m_initial, b_initial, x_values, y_values, learn_rate, steps):
    m = m_initial
    b = b_initial
    for i in range(steps):
        (m, b) = step(m, b, x_values, y_values, learn_rate)
    return (m, b)

In [26]:
scale = 1/(2*max(x_vals)**2) # This is basically the invese of bound on the Laplacian of the error
(m,b) = learn(1, 25, x_vals, y_vals, scale*0.1, 1000000)

In [27]:
(m,b)

(9.774389416905849e-05, 24.999992472546513)

In [28]:
def line_trace(m,b):
    xs = np.linspace(0.0, max(x_vals), 100)
    ys = m*xs + b
    return {'x' : list(xs), 'y' : list(ys), 'mode' : 'lines'}

In [29]:
figure2 = {'data': [trace, line_trace(m,b)], 'layout': {'title': 'STEC Incidence Rate versus Cattle Population'}}
plot(figure2)

This looks a bit unconvincing. How can we quantify how good a fit this is? Let's try the Normalized Root Mean Square Error:

In [30]:
RMSE(m, b, x_vals, y_vals) / (max(y_vals) - min(y_vals))

0.12560298881135756

This is alright (around 12% error rate). This indicates there are likely some variables which we have not accounted for. If we wanted to explore this further, we could compare the cloropleths of STEC outbreaks in Texas versus cattle population and see which counties don't fit the general trend:

In [31]:
#fips = list(texasCattle['COUNTY_CODE'].apply(str))  # Read off county codes
#fips = list(map(lambda s: s.zfill(3), fips))        # Prefix leading zeros
#fips = list(map(lambda code: '48' + code, fips))    # Prefix Texas state code 48 
fips2 = list(texasSTEC['FIPS'])
values2 = list(texasSTEC['AGG_RATE'])
#values = list(map(lambda s: s.replace(',',''), values))
#values = list(map(int, values))
endpts = list(np.mgrid[min(values2):max(values2):7j])
print(endpts)
#endpts = list(map(lambda x: round(x, 0), endpts))
colorscale = valsToColors(endpts, [200,255,0], [255,50,0])
print(colorscale)
endpts.pop(0)
#endpts.pop(-1)

fig2 = ff.create_choropleth(fips=fips2, values=values2, scope=['TX'],
                           binning_endpoints=endpts, colorscale=colorscale,
                           county_outline={'color': 'rgb(0,0,0)', 'width': 0.5},
                           legend_title='STEC cases per decade per 100,000 inhabitants')

fig2.layout.template = None
fig2.show()

[0.0, 39.96666666666666, 79.93333333333332, 119.89999999999998, 159.86666666666665, 199.83333333333331, 239.79999999999995]
['rgb(200, 255, 0)', 'rgb(209, 221, 0)', 'rgb(218, 187, 0)', 'rgb(228, 152, 0)', 'rgb(237, 118, 0)', 'rgb(246, 84, 0)', 'rgb(255, 50, 0)']


Compare this to our previous cloropleth for number of cattle:

In [32]:
fig.show()

We can see there are significantly more STEC outbreaks in West Texas than our simple model predicts, so this would be one place to look next. It may be there is a different type of animal being raised there, or some other factor in STEC cases we are not considering. 